<a href="https://colab.research.google.com/github/Himanshu-kashyap/Tutorials/blob/master/bs_reddit_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## scraping old.reddit.com 

In [0]:
import urllib.request
from bs4 import BeautifulSoup
import json

In [0]:
# div id = sitetable contains all the data
url = "https://old.reddit.com/"
request = urllib.request.Request(url)
html = urllib.request.urlopen(request).read()

In [0]:
# provide html and parser to BS
soup = BeautifulSoup(html,'html.parser')    # can also be lxml instead of html parser
main_table = soup.find("div",attrs = {'id':'siteTable'})  # div id = siteTable contains everything
links = main_table.find_all("a",class_ = "title")  # returns a result set

In [0]:
# make a list of dictionaries with text title and url; if url is not starting with html, make it.

extracted_records = []
for link in links:
  title = link.text
  url_ = link['href']
  if not url_.startswith("html"):
    url_ = "https://old.reddit.com"+url_
  record = {
      'title':title,
      'url':url_
  }
  extracted_records.append(record)

In [0]:
print(extracted_records)

[{'title': 'AITA for telling cashier that wasn’t the girls credit card?', 'url': 'https://old.reddit.com/r/AmItheAsshole/comments/cdypge/aita_for_telling_cashier_that_wasnt_the_girls/'}, {'title': 'Not mine but holy stupidity', 'url': 'https://old.reddit.com/r/ChoosingBeggars/comments/ce3d8f/not_mine_but_holy_stupidity/'}, {'title': 'When pulling a Rabbit out of a hat goes wrong', 'url': 'https://old.reddit.comhttps://gfycat.com/immaculategrouchyamericanbadger'}, {'title': 'Redditors with interesting hobbies- what do you do and why?', 'url': 'https://old.reddit.com/r/AskReddit/comments/cdwvhh/redditors_with_interesting_hobbies_what_do_you_do/'}, {'title': 'America’s Finest.', 'url': 'https://old.reddit.com/r/PublicFreakout/comments/cdwuat/americas_finest/'}, {'title': 'Kellyanne Conway Snaps Back at Reporter: ‘What’s Your Ethnicity?’', 'url': 'https://old.reddit.comhttps://www.thedailybeast.com/kellyanne-conway-snaps-back-at-reporter-whats-your-ethnicity?source=politics&via=rss'}, {'ti

## dump as json


In [0]:
with open('data.json','w') as outfile:
  json.dump(extracted_records,outfile)

## Comments and beyond



In [0]:
url = "https://old.reddit.com/"
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
request = urllib.request.Request(url,headers={'User-Agent': user_agent})
html = urllib.request.urlopen(request).read()
soup = BeautifulSoup(html,'html.parser')
#First lets get the HTML of the table called site Table where all the links are displayed

main_table = soup.find("div",attrs={'id':'siteTable'})

In [0]:
comment_a_tags = main_table.find_all("a",attrs = {"class": "bylink comments may-blank"})

In [0]:
urls = [] 
for a_tag in comment_a_tags:
    url = a_tag['href']
    if not url.startswith('http'):
        url = "https://reddit.com"+url
    urls.append(url)

### Take any comment and find the items you want

In [0]:
urls[0]

'https://old.reddit.com/r/AmItheAsshole/comments/cdypge/aita_for_telling_cashier_that_wasnt_the_girls/'

we will find
* Title
* upvotes,
* original_poster,
* comments_count,
* extracted_comments

In [0]:
def url_parser(page_url):
  user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
  request = urllib.request.Request(page_url,headers={'User-Agent': user_agent})
  html = urllib.request.urlopen(request).read()
  soup = BeautifulSoup(html,'html.parser')
  
  main_post = soup.find("div",attrs = {'id':'siteTable'})
  title = main_post.find("a",attrs = {'class':"title"})
  upvotes = main_post.find("div",attrs = {'class':'unvoted'})
  original_poster = main_post.find('a',attrs={'class':'author'}).text
  comments_count = main_post.find('a',attrs={'class':'bylink comments may-blank'}).text
  comment_area = soup.find('div',attrs={'class':'commentarea'})
  comments = comment_area.find_all('div', attrs={'class':'entry unvoted'})
  extracted_comments = []
  for comment in comments: 
        if comment.find('form'):
            #We are now looking for any element with a class of author in the comment, instead of just looking for a tags. 
            #We noticed some comments whose authors have deleted their account shows up with a span tag instead of an a 
            commenter = comment.find(attrs={'class':'author'}).text
            comment_text = comment.find('div',attrs={'class':'md'}).text.strip()
            permalink = comment.find('a',attrs={'class':'bylink'})['href']
            extracted_comments.append({'commenter':commenter,'comment_text':comment_text,'permalink':permalink})
    #Lets put the data in dict 
  post_data = {
        'title':str(title),
        'no_of_upvotes':str(upvotes),
        'poster':str(original_poster),
        'no_of_comments':str(comments_count),
        'comments':str(extracted_comments)
    }
  return post_data

In [0]:
json_x = url_parser("https://old.reddit.com/r/AdviceAnimals/comments/ce5yv1/at_work_with_a_coworker_in_a_group_conversation/")

In [16]:
type(json_x)

dict

In [0]:
with open('comment1.json','w') as jsonfile:
  json.dump(json_x,jsonfile)

In [0]:
soup